In [3]:
!pip install 'apache-beam' 
!pip install "apache-beam[gcp,interactive,dataframe]" 
!pip install text2emotion
!pip install stanza


ERROR: Invalid requirement: "'apache-beam'"


     ---------------------------------------- 0.0/89.7 kB ? eta -:--:--
     ------------------ --------------------- 41.0/89.7 kB 1.9 MB/s eta 0:00:01
     ---------------------------------- --- 81.9/89.7 kB 919.0 kB/s eta 0:00:01
     -------------------------------------- 89.7/89.7 kB 845.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/50.7 kB ? eta -:--:--
     ---------------------------------------- 50.7/50.7 kB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/152.0 kB ? eta -:--:--
     --------- ------------

In [4]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.window import GlobalWindows
import text2emotion as te
import stanza
import time
from datetime import date

# Initialize your stanza and text_emotion models
nlp = stanza.Pipeline()



# Initialize your stanza and text_emotion models
nlp = stanza.Pipeline()

pipeline_options = PipelineOptions()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\volte\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\volte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\volte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-03-28 10:28:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-03-28 10:28:08 INFO: Downloaded file to C:\Users\volte\stanza_resources\resources.json
2024-03-28 10:28:10 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-03-28 10:28:10 INFO: Using device: cpu
2024-03-28 10:28:10 INFO: Loading: tokenize
2024-03-28 10:28:13 INFO: Loading: mwt
2024-03-28 10:28:13 INFO: Loading: pos
2024-03-28 10:28:14 INFO: Loading: lemma
2024-03-28 10:28:14 INFO: Loading: constituency
2024-03-28 10:28:15 INFO: Loading: depparse
2024-03-28 10:28:16 INFO: Loading: sentiment
2024-03-28 10:28:16 INFO: Loading: ner

2024-03-28 10:28:18 INFO: Downloaded file to C:\Users\volte\stanza_resources\resources.json
2024-03-28 10:28:21 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-03-28 10:28:21 INFO: Using device: cpu
2024-03-28 10:28:21 INFO: Loading: tokenize
2024-03-28 10:28:21 INFO: Loading: mwt
2024-03-28 10:28:21 INFO: Loading: pos
2024-03-28 10:28:21 INFO: Loading: lemma
2024-03-28 10:28:22 INFO: Loading: constituency
2024-03-28 10:28:23 INFO: Loading: depparse
2024-03-28 10:28:24 INFO: Loading: sentiment
2024-03-28 10:28:25 INFO: Loading: ner

In [ ]:
def classify_text(text):
    doc = nlp(text)
    sentiment_score = sum([sentence.sentiment for sentence in doc.sentences]) / len(doc.sentences)
    topics = list()
    for sentence in doc.sentences:
        for entity in sentence.ents:
            if entity.type != 'O':  # Exclude non-entity tokens
                topics.append(entity.text)
    topics = ",".join(topics)
    emotions = te.get_emotion(text)
    return [sentiment_score, topics, emotions]

class ClassifyTextDoFn(beam.DoFn):
    def process(self, element):
        text = element['text']  # Replace 'your_text_column_name' with the actual column name containing the text
        
        # Use the shared nlp and te objects
        doc = nlp(text)
        
        sentiment_score = sum([sentence.sentiment for sentence in doc.sentences]) / len(doc.sentences)
        
        topics = list()
        for sentence in doc.sentences:
            for entity in sentence.ents:
                if entity.type != 'O':  # Exclude non-entity tokens
                    topics.append(entity.text)
        topics = ",".join(topics)
        
        emotions = te.get_emotion(text)
        
        # Yield each value separately
        yield {
            'comment': text,
            'sentiment': sentiment_score,
            'topic': topics,
            'happiness': emotions["Happy"],
            'anger': emotions["Angry"],
            'sadness': emotions["Sad"],
            'fear': emotions["Fear"]
        }



pipeline_options = beam.options.pipeline_options.PipelineOptions(
    temp_location='gs://reddit_raw_data_0184598608709384596/temp/',
    project='ml-deployments-practice',
    region="europe-west1",
    runner="DataflowRunner",
    job_name="classify-reddit-comments",  # Unique job name for Dataflow
    num_workers=10,  # Number of worker nodes
    max_num_workers=20,  # Maximum number of worker nodes
    autoscaling_algorithm='THROUGHPUT_BASED',  # Autoscaling algorithm
    machine_type='n1-standard-4'  # Machine type
)

start = time.time()
today = str(date.today())
with beam.Pipeline(options=pipeline_options) as p:
    
    query = f"SELECT * FROM reddit_comment_data.raw_daily_hot_comments WHERE date= {today}"
    
    data = (
        p
        | 'Read from BigQuery' >> beam.io.Read(beam.io.ReadFromBigQuery(query=query))
        | 'Classify text' >> beam.ParDo(ClassifyTextDoFn())
        | 'Write to BigQuery' >> beam.io.WriteToBigQuery(
            table='ml-deployments-practice.reddit_comment_data.beam_test_output',
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
            custom_gcs_temp_location='gs://reddit_raw_data_0184598608709384596/temp/'
        )
    )
    
end = time.time()
print(end - start)